In [ ]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [ ]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump, symb_perf_stats_vectorized

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

verbose = False  # True prints more output
look_back_days = -250 * 6  # subset df iloc days

In [ ]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

In [ ]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_OHLCVA[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

In [ ]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

In [ ]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume)  # combine symbols with no volume and no close
print(f'symbols_drop with duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
print(f'symbols_drop with unique symbols: {len(symbols_drop)}')

In [ ]:
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_a

In [ ]:
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
df_c

In [ ]:
print(f'symbols with no volume: {len(symbols_no_volume)}')
print(f'symbols with no close: {len(symbols_no_close)}')
print(f'unique symbols dropped from df_a and df_c: {len(symbols_drop)}')

In [ ]:
# _periods = [-15, -30, -60, -120, -240]

perf_rank_dict = {}
syms_perf_rank = []
_periods = [-15, -30, -60, -120, -240]
for _period in _periods:
  f_name = 'period' + str(_period)

  _df_c = df_c[_period::]
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(_df_c)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']

  # write symbols' performance stats to dataframe
  df_ps = pd.DataFrame(caches_perf_stats_vect, columns=column_names)
  df_ps['r_CAGR/UI'] = df_ps['CAGR/UI'].rank(ascending=False)
  df_ps['r_CAGR/Std'] = df_ps['CAGR/Std'].rank(ascending=False)
  df_ps['r_Std/UI'] = df_ps['Std/UI'].rank(ascending=False)
  
  _dict = {}
  cols_sort = ['r_CAGR/UI', 'r_CAGR/Std', 'r_Std/UI']
  print(f'{f_name} top 10 symbols')  
  for col in cols_sort:
    symbols_top_10 = df_ps.sort_values(by=[col]).head(10).symbol.values
    syms_perf_rank.append(list(symbols_top_10))
    print(f'{col}: {symbols_top_10}')
    _dict[col] = symbols_top_10
    perf_rank_dict[f'{f_name}'] = _dict
  print(' ')

pickle_dump(perf_rank_dict, path_data_dump, 'perf_rank_dict')
print(f'Pickled perf_rank_dict to: {path_data_dump}perf_rank_dict\n')
print(f'perf_rank_dict:\n{perf_rank_dict}\n')
print(f'syms_perf_rank: {syms_perf_rank}')


In [ ]:
syms_perf_rank  # list of lists
l_syms_perf_rank = [val for sublist in syms_perf_rank for val in sublist]  # flatten list of lists

from collections import Counter
c = Counter(l_syms_perf_rank)  # count symbols and count
print(c) 
c_tuples = c.most_common(13)[3:]  # e.g [('AKRO', 6), ('IMVT', 4), ... ('ADEA', 3)]
l_symbols = [symbol for symbol, count in c_tuples]  # top 3-13 most common symbols in syms_perf_rank
l_symbols  